# 01 Data Preparation

Load and clean the OSM-derived grid data, then package it for remote runs.


In [1]:
from pathlib import Path
import sys
import os
import pypsa
import numpy as np
import pandas as pd


def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()
src_path = repo_root / 'src/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))
print(f"Using src path: {src_path}")
print(f"Repository root: {repo_root}")

import pypsa_simplified as ps

src_path = repo_root / 'scripts/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))

import geometry as geom

Using src path: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/src
Repository root: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model


## Data sources and parsing
- **OSM prebuilt electricity network** (`data/raw/OSM Prebuilt Electricity Network/`): buses, lines, links, converters, transformers.
- **Custom CSV parsing**: `prepare_osm_source` uses a geometry-safe loader (handles commas inside WKT) to keep column counts correct.
- **Endpoint extraction**: First/last coordinates are pulled from WKT to map line/link endpoints to buses (tolerance 1e-5 degrees).
- **Country filter**: Defaults to DE/FR/PL/AT/IT; adjust via `countries` if needed.


In [2]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
from pypsa_simplified import prepare_osm_source
from pypsa_simplified import prepare_generator_data


osm_dir = repo_root / "data" / "raw" / "OSM Prebuilt Electricity Network"
sources = prepare_osm_source(osm_dir)
print({k: v.shape if hasattr(v, 'shape') else v for k, v in sources.items()})

RawData = ps.data_prep.RawData(sources)

source = prepare_generator_data(repo_root / "data" / "raw" / "powerplants.csv")

{'buses': (6737, 10), 'lines': (8994, 16), 'converters': (67, 7), 'links': (38, 10), 'transformers': (875, 8), 'generators': None, 'loads': None, 'storage': None}


In [3]:
EU27 = [
    'AT', 'BE', 'BG', 'HR', 'CZ', 'DK', 'EE', 'EL', 'FI', 'FR',
    'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL',
    'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'
] + ['UK', 'CH', 'NO', 'GB']  # include UK, CH, NO for practical purposes

In [4]:
RawData.data['buses']

,bus_id,voltage,dc,symbol,under_construction,tags,x,y,country,geometry
0,AL1-220,220.0,f,Substation,f,AL1,20.111702,42.096798,AL,POINT (20.111702399999995 42.09679778825765)
1,AL10-220,220.0,f,Substation,f,AL10,19.522149,40.478444,AL,POINT (19.522149214338864 40.47844442711058)
2,AL2-220,220.0,f,Substation,f,AL2,20.031168,42.069050,AL,POINT (20.031168199999993 42.06905038823308)
3,AL3-400,400.0,f,Substation,f,AL3,19.652495,42.010831,AL,POINT (19.652494799999996 42.01083068818158)
4,AL4-220,220.0,f,Substation,f,AL4,19.972850,41.584771,AL,POINT (19.9728498 41.58477058780021)
...,...,...,...,...,...,...,...,...,...,...
6732,way/99694910-380,380.0,f,Substation,f,way/99694910,7.789278,44.462732,IT,POINT (7.7892782005302115 44.46273179679329)
6733,way/99722046-225,225.0,f,Substation,f,way/99722046,-0.114171,47.206984,FR,POINT (-0.1141712110057602 47.206983706128305)
6734,way/99722046-400,400.0,f,Substation,f,way/99722046,-0.113930,47.207199,FR,POINT (-0.113930394824857 47.20719863601106)
6735,way/99826025-220,220.0,f,Substation,f,way/99826025,7.785444,44.875461,IT,POINT (7.785443587366926 44.87546070934813)


## Create a network
Use the RawData to create the first network

In [5]:
snapshots = pd.date_range("2024-01-01", "2024-12-31 23:00", freq="h")
NETWORK_METADATA = {
    'name': 'Simplified European Electricity Network',
    'snapshots': snapshots,
    'countries': EU27,
    'generation_carriers': None,
    'transmission_carriers': None,
    'join': True,
}

In [6]:
n = pypsa.Network()
n = ps.build_network(n, RawData, options=NETWORK_METADATA)

Adding 6295 buses...
Adding 8385 lines...
Adding 812 transformers...
Adding 67 converters...
Adding 37 links...


In [7]:
n

PyPSA Network 'Simplified European Electricity Network'
-------------------------------------------------------
Components:
 - Bus: 6359
 - Carrier: 15
 - Line: 8385
 - Link: 103
 - Transformer: 812
Snapshots: 8784

In [8]:
network_path = repo_root / "data" / "processed" / "networks" / "simplified_european_network.nc"
#ps.save_netwok(n, repo_root / "data" / "processed" / "neworks" / "simplified_european_network.nc")
n.export_to_netcdf(network_path)

INFO:pypsa.network.io:Exported network 'Simplified European Electricity Network' saved to '/Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/networks/simplified_european_network.nc contains: links, buses, transformers, carriers, lines


<xarray.Dataset> Size: 1MB
Dimensions:                   (snapshots: 8784, links_i: 103, buses_i: 6359,
                               transformers_i: 812, carriers_i: 15,
                               lines_i: 8385)
Coordinates:
  * snapshots                 (snapshots) int64 70kB 0 1 2 3 ... 8781 8782 8783
  * links_i                   (links_i) object 824B 'conv-relation/10629005' ...
  * buses_i                   (buses_i) object 51kB 'AT1-220' ... 'way/128472...
  * transformers_i            (transformers_i) object 6kB 'BG10-220-400' ... ...
  * carriers_i                (carriers_i) object 120B 'hard coal' ... 'DC'
  * lines_i                   (lines_i) object 67kB 'merged_relation/10264161...
Data variables: (12/33)
    snapshots_snapshot        (snapshots) datetime64[ns] 70kB 2024-01-01 ... ...
    snapshots_objective       (snapshots) float64 70kB 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_stores          (snapshots) float64 70kB 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_generators      (snapshots) float64 70kB 1.0 1.0 1.0 ... 1.0 1.0
    links_bus0                (links_i) object 824B 'relation/10629005' ... '...
    links_bus1                (links_i) object 824B 'relation/10629005-400-DC...
    ...                        ...
    lines_r                   (lines_i) float64 67kB 0.3554 1.777 ... 0.3144
    lines_b                   (lines_i) float64 67kB 2.326e-05 ... 2.057e-05
    lines_s_nom               (lines_i) float64 67kB 502.7 1.787e+03 ... 491.6
    lines_length              (lines_i) float64 67kB 5.923e+03 ... 5.239e+03
    lines_carrier             (lines_i) object 67kB 'AC' 'AC' 'AC' ... 'AC' 'AC'
    lines_num_parallel        (lines_i) float64 67kB 1.0 1.0 1.0 ... 2.0 1.0 1.0
Attributes:
    network_name:           Simplified European Electricity Network
    network_pypsa_version:  1.0.5
    network_srid:           4326
    crs:                    {"_crs": "GEOGCRS[\"WGS 84\",ENSEMBLE[\"World Geo...
    meta:                   {}

In [9]:
n = pypsa.Network(network_path)

INFO:pypsa.network.io:Imported network 'Simplified European Electricity Network' has buses, carriers, lines, links, transformers


In [10]:
n

PyPSA Network 'Simplified European Electricity Network'
-------------------------------------------------------
Components:
 - Bus: 6359
 - Carrier: 15
 - Line: 8385
 - Link: 103
 - Transformer: 812
Snapshots: 8784

In [11]:
n = ps.add_loads_from_series(n)

Adding demand for 6258 buses...
Load prep failed for bus AT1-220: Demand series for bus AT1-220 must be a list of floats.
Load prep failed for bus AT10-220: Demand series for bus AT10-220 must be a list of floats.
Load prep failed for bus AT11-220: Demand series for bus AT11-220 must be a list of floats.
Load prep failed for bus AT12-380: Demand series for bus AT12-380 must be a list of floats.
Load prep failed for bus AT13-380: Demand series for bus AT13-380 must be a list of floats.
Encountered 6258 errors while preparing/adding loads.
Added 0 loads with time series.


In [ ]:
n.export_to_netcdf(network_path)

In [8]:
load_path = repo_root / "data" / "processed" / "voronoi_demand_join.gzip"
load_data = pd.read_parquet(load_path)


In [3]:
processed_demand = pd.read_csv(repo_root / "data" / "raw" / "energy_charts" / "processed_load_data.csv", index_col=0, parse_dates=True)

/var/folders/y8/4_9g68pj7k136q2yypgp5ysc0000gn/T/ipykernel_42103/4212074812.py:1: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [9]:
processed_demand.groupby('Timestamp')['Load'].sum().max() 

523319.525

In [6]:
processed_demand = processed_demand.sort_values(by= ['Country','Timestamp'])

In [7]:
display(processed_demand)

,Country,Load,Missing
Timestamp,,,
2015-01-01 00:00:00+00:00,AT,6017.20,False
2015-01-01 01:00:00+00:00,AT,5750.80,False
2015-01-01 02:00:00+00:00,AT,5407.20,False
2015-01-01 03:00:00+00:00,AT,5284.80,False
2015-01-01 04:00:00+00:00,AT,5280.40,False
...,...,...,...
2024-12-31 19:00:00+00:00,UK,41563.34,False
2024-12-31 20:00:00+00:00,UK,38654.07,False
2024-12-31 21:00:00+00:00,UK,36124.27,False


In [ ]:
processed_demand[(processed_demand['Country']=='AT') & (processed_demand.index.year < 2016)].sum() # 59235762.2

Country    ATATATATATATATATATATATATATATATATATATATATATATAT...
Load                                              59235762.2
Missing                                                    0
dtype: object

### Next
- Use `notebooks/main.ipynb` to transfer the artifact and trigger the remote optimization.
- For custom country lists or tolerance, pass `countries`/`tol` to `prepare_osm_source`.
